In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import pickle
import matplotlib.pyplot as plt
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn import metrics
from sklearn.metrics import classification_report



C:\Users\kelse\Anaconda3\new\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\kelse\Anaconda3\new\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https:/

In [5]:
#Set Random seed
np.random.seed(500)

# Import Data
df = pd.read_csv(r"C:\Users\kelse\OneDrive\Documents\GitHub\HTMLElementClassifier\Dataset",encoding = "ISO-8859-1")
df.head()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\kelse\\OneDrive\\Documents\\GitHub\\HTMLElementClassifier\\Dataset'

In [ ]:
#Size of dataset
df.shape
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [ ]:
df['HTMLElement'].dropna(inplace=True)
df['Category'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.ylabel("Number of Samples")
plt.xlabel("Categories")

In [ ]:
df = df.astype(str)

In [ ]:
df['HTMLElement'] = [entry.lower() for entry in df['HTMLElement']] #change lowercase
df['HTMLElement']= [word_tokenize(entry) for entry in df['HTMLElement']] #perform tokenization

# lemmantizazing
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


for index,entry in enumerate(df['HTMLElement']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)
    
print(df['text_final'])

In [ ]:
#Split model
x_Train, x_Test, y_Train, y_Test = model_selection.train_test_split(df['text_final'].astype(str),df['Category'],test_size=0.33)

encoder = LabelEncoder()
y_Train = encoder.fit_transform(y_Train.astype(str))
y_Test = encoder.fit_transform(y_Train.astype(str))

In [ ]:
#BagOfWords Approach 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['text_final'].astype(str))
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
print(df_bow_sklearn.head())

Train_X = vectorizer.transform(x_Train)
Test_X = vectorizer.transform(x_Test)

Train_X.reshape(-1,1)
Test_X.reshape(-1,1)

Train_X.shape


In [ ]:
#Multinomial Naive Bayes
nb = naive_bayes.MultinomialNB()
nb.fit(Train_X, y_Train)
predictions_NB = nb.predict(Train_X)

In [ ]:
#Multiclass
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
clf = OneVsOneClassifier( LinearSVC(random_state=0)).fit(Train_X_Tfidf, Train_Y)
predictions_SVM = clf.predict(Test_X_Tfidf)
print("Precision:",metrics.precision_score(Test_Y, predictions_SVM))
print("Recall:",metrics.recall_score(Test_Y, predictions_SVM))

In [ ]:
#SVM_Classifier

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X,y_Train)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X)
#print("Precision:",metrics.precision_score(y_Test, predictions_SVM))
#print("Recall:",metrics.recall_score(y_Test, predictions_SVM))
predictions_SVM

In [ ]:
##Confusion Matrix
from sklearn.metrics import confusion_matrix

#Generate the confusion matrix
cf_matrix = confusion_matrix(y_Test, predictions_SVM)

print(cf_matrix)

import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

print (classification_report(y_Test, predictions_SVM))

In [ ]:
#Testing the Classification

test = ['<input class="quer" type="text" value="" style="border: solid #b9b9b9 2px ; height: 3em; width: 78%; border-radius:10px; padding-left: 8%; font-size: 1.5em; font-weight: 600; outline: none; margin-top: 2%" placeholder="Search">']
stop_words = set(stopwords.words("english"))

element = [entry.lower() for entry in test]

# Step - 1c : Tokenization : In this each entry in the corpus will be broken into set of words
element= [word_tokenize(entry) for entry in test]
lemmatizer = WordNetLemmatizer()
#element = [lemmatizer.lemmatize(word) for word in element if not word in set(stop_words)]

final_words = []
for word in element:
    if word not in stopwords.words('english'):
        final_words.append(word)
        
test_processed =[ ' '.join([lemmatizer.lemmatize(word) for word_list in element for word in word_list])]
test_processed

#test_input = Tfidf_vect.transform(test_processed)
test_input = vectorizer.transform(test_processed)
test_input.shape


res= SVM.predict(test_input)[0]
print(res,test_processed)
if res==1:
    print("Search")
elif res==0:
    print("AddToCart")
elif res==2:
    print("Checkout")
